In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import plotly_express as px
import warnings
import collections
warnings.filterwarnings("ignore")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity



import os
import pymysql
from sqlalchemy import create_engine



import nltk
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords



In [ ]:
from nltk.tokenize import word_tokenize
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('punkt')


#tokens = word_tokenize(text)
#tokens[:15]
# stemming cleaning technique -> treats prefixes and suffixes. NOT USED
from nltk.stem import PorterStemmer

ps = PorterStemmer()
# stemmed = [ps.stem(w) for w in tokens]
#stemmed[:15]

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
nltk.download('wordnet') # wordnet is the most well known lemmatizer for english
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer() 
#lemmatized = [lemmatizer.lemmatize(word) for word in tokens]
#lemmatized[:15]

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


##  <font color='CornflowerBlue'><center><strong>I. <ins>Doing some more cleaning - Stop Words </ins> </strong><center><font color='blue'></font>


In [47]:
df = pd.read_csv("recipes_10_lem.csv", index_col=0)

In [ ]:
df["lemmatized"] = df["lemmatized"].str.replace("', '"," " ).str.replace("['", "").str.replace("']", "").str.replace("nan ", "")
df["lemmatized"] = df["lemmatized"].str.replace("nan", "") #clean the string transform into a list

In [56]:
df["lemmatized"] = df["lemmatized"].str.split(" ") #transform into a list

In [52]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rober\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [60]:
#df["lemmatized2"] = df["lemmatized"].apply(lambda lemmatized: [word for word in lemmatized if not word in stopwords.words()])
#df.to_csv("recipes_10_lem.csv", index=False)

# this took 625 m to run

In [3]:
new_df = pd.read_csv("recipes_10_lem.csv")

In [ ]:
type(new_df["lemmatized2"][0])

new_df["lem_list"]  = new_df["lemmatized2"].str.replace("', '"," " ).str.replace("['", "").str.replace("']", "").str.replace("nan ", "") #clean the string transform into a list
#new_df["lem_list"] = new_df["lem_list"].str.replace("nan", "") 
new_df = new_df.rename(columns={"lem_list": "lems_string", "lemmatized2": "no_stop_word"})
new_df = new_df.drop(columns="lem_str")


new_df.to_csv("recipes_10_lem.csv", index=False)
new_df.head()

In [4]:

df_f = df[(df["website_id"] != 2)]

In [61]:
df_f["website_id"].value_counts() 

website_id
3    494963
1       573
4       285
Name: count, dtype: int64

In [ ]:
#dfs.to_csv("recipes_10_lem.csv")
##dfs = pd.read_csv("recipes_10_lem.csv")
#dfs.shape
#dfs.reset_index(inplace=True)
dfs = df.reset_index(drop=True)
dfs.head(2)


In [17]:
def user_recipe():
    user_recipe = input("Give me a recipe that you like:")
    return user_recipe


In [ ]:
cv_char = CountVectorizer(analyzer='char_wb', ngram_range = (2,2))
list_recipes = list(dfs["recipe"])
#user_recipe= user_recipe()
user_recipe = 'ratatouile' 
list_recipes.append(user_recipe)
count_matrix = cv_char.fit_transform(list_recipes)

In [ ]:
count_matrix = cv_char.fit_transform(list_recipes)
cosine_sim = cosine_similarity(count_matrix)
similar_recipes = list(enumerate(cosine_sim[-1]))
sorted_similar_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
suggested_id = [d[0] for d in sorted_similar_recipes[:1]] # returns just one recipe
re_user = dfs.loc[suggested_id]
print(suggested_id)
print(re_user)
#for x in suggested_id:
    #print(str(dfs["recipe"][x]))

### inputing the recipe

vege_df = dfs[(dfs["diet_id"] == 1) | (dfs["diet_id"] == 2)]
#vege_df.head(1)
#vege_df["diet_id"].value_counts()
#vege_df["website_id"].value_counts()
#vege_df.shape
#vege_df.to_csv("vege_df.csv")
#vege_df = pd.read_csv("vege_df.csv")
cv = CountVectorizer()
#re_user = dfs.loc[suggested_id]
df_search = pd.concat([vege_df, re_user], axis=0)
df_search = df_search.reset_index(drop=True)
list_features = list(df_search["lemmatized2"])
count_matrix = cv.fit_transform(list_features)

cosine_sim = cosine_similarity(count_matrix)
similar_recipes = list(enumerate(cosine_sim[-1]))
sorted_similar_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
suggested_ids = [d[0] for d in sorted_similar_recipes[:5]]
final_recipes = df_search.loc[suggested_ids]
final_recipes

In [60]:
list_features[1861]

'['

In [79]:
final_recipes["list_ingredients"][552]

"['1                                                                                                                                                                                                                     butternut squash                                                                                                                                                                , (1.2kg)', '250                                                                                                                                                                g                                                                                                                                                                ricotta cheese', '½                                                                                                                                                                teaspoon                                                                                   

In [73]:
final_recipes["recipe"][570]

'easy veggie stir-fry'

In [74]:
final_recipes["recipe"][490]

'vegan mushroom, chestnut & cranberry tart'

In [ ]:
final_recipes["list_ingredients"][570]

In [ ]:
final_recipes["list_ingredients"][490]

### Final Fonction:

In [ ]:
#cv_char = CountVectorizer(analyzer='char_wb', ngram_range = (2,2))
list_recipes = list(dfs["recipe"])

In [ ]:
#cv = CountVectorizer()

In [ ]:
def user_recipe():
    '''gives the user an input , returns the result'''
    user_recipe = input("Give me a recipe that you like:")
    return user_recipe

In [ ]:
def search_recipe_name(user_recipe, list_recipes):
    '''takes the result of the input , as well as the list of recipes of the database
    user returns a df with the row containg the user recipe or the closest to it'''
    list_recipes = list_recipes.append(user_recipe)
    count_matrix = cv_char.fit_transform(list_recipes)
    cosine_sim = cosine_similarity(count_matrix)
    similar_recipes = list(enumerate(cosine_sim[-1]))
    sorted_similar_recipes = sorted(similar_recipes,key=lambda x:x[1],reverse=True)[1:]
    suggested_id = [d[0] for d in sorted_similar_recipes[:1]]
    re_user = dfs.loc[suggested_id]
    return re_user

In [ ]:
def search_final_recipe(vege_df, re_user):
    cv = CountVectorizer()
    df_search = pd.concat(vege_df, re_user, axis=0)
    df_search = df_search.reset_index(drop=True)
    list_features = list(df_search["lemmatized"])
    

In [ ]:
def print_recipe(suggested_ids):
    print(f"Top similar recipes to {user_recipe}")
    for x in suggested_ids:
        final_recipes.loc[x]
        name = final_recipes["recipe"]
        ingredients = final_recipes["list_ingredients"]
        steps = final_recipes["list_instructions"]
        if name and ingredients and steps: 
            print(f"Recipe: {name}")
            print(f"Ingredients: {ingredients}")
            print(f"Steps: {steps}")
        